In [1]:
import pandas as pd
import json
import numpy as np
import statsmodels.stats.multitest as smm

In [9]:
def get_df(folder, mean=True):
    json_files = ['fold1/history.json', 'fold2/history.json', 
                  'fold3/history.json', 'fold4/history.json', 'fold5/history.json']

    metrics = ['loss', 'mean_rank', 'median_rank', 'recall@1', 'recall@5', 'recall@10', 'recall@50']
    categories = ['Training', 'Validation']

    # Initialisiere Dict für die letzten Werte
    last_values = {cat: {m: [] for m in metrics} for cat in categories}

    for json_file in json_files:
        with open(f"{folder}/{json_file}", 'r') as f:
            data = json.load(f)
            for cat in categories:
                for metric in metrics:
                    last_values[cat][metric].append(data[cat][metric][-1])

    if mean:
        # Mittelwerte berechnen
        result = {cat: {metric: np.mean(values) for metric, values in metrics.items()} for cat, metrics in last_values.items()}
    else:
        # Rohwerte beibehalten
        result = last_values

    return pd.DataFrame(result)

In [10]:
get_df("vitb16_2_humanedit_freeze_none", mean=True)

,Training,Validation
loss,8.388089,8.392132
mean_rank,140.340878,161.218044
median_rank,21.600000,29.600000
recall@1,11.554054,9.190556
recall@5,28.720439,24.131535
recall@10,38.382601,33.423272
recall@50,63.754223,59.494098


In [11]:
get_df("vitb16_2_magicbrush_freeze_none", mean=True)

,Training,Validation
loss,9.129075,9.130819
mean_rank,355.468693,369.143928
median_rank,60.200000,77.800000
recall@1,8.017045,6.258461
recall@5,18.710227,15.197954
recall@10,25.196023,21.388350
recall@50,47.102273,42.537922


In [12]:
vitb16_2_humanedit = get_df("vitb16_2_humanedit_freeze_none", mean=False)
vitb16_2_magicbrush = get_df("vitb16_2_magicbrush_freeze_none", mean=False)
vitb16_8_humanedit = get_df("vitb16_8_humanedit_freeze_none", mean=False)
vitb32_2_humanedit = get_df("vitb32_2_humanedit_freeze_none", mean=False)

In [6]:
vitb16_2_humanedit

,Training,Validation
loss,"[8.387917141656619, 8.387737731675845, 8.38714...","[8.39388031822835, 8.394403858892431, 8.393683..."
mean_rank,"[138.0785472972973, 141.31186655405406, 141.15...","[165.67622259696458, 152.83558178752108, 162.2..."
median_rank,"[22, 22, 21, 21, 22]","[33, 28, 32, 28, 27]"
recall@1,"[11.465371621621621, 11.84543918918919, 11.634...","[8.347386172006747, 8.263069139966273, 9.44350..."
recall@5,"[28.631756756756754, 29.43412162162162, 28.821...","[24.283305227655987, 22.17537942664418, 23.693..."
recall@10,"[38.09121621621622, 38.38682432432432, 38.7246...","[33.473861720067454, 31.703204047217536, 33.22..."
recall@50,"[63.914695945945944, 63.36570945945946, 63.893...","[57.67284991568297, 59.86509274873525, 58.4317..."


In [13]:
from scipy.stats import ttest_ind
from scipy.stats import ttest_ind
import statsmodels.stats.multitest as smm

# Liste der Vergleichs-Datenframes
comparisons = {
    "vitb16_2_magicbrush": vitb16_2_magicbrush,
    "vitb16_8_humanedit": vitb16_8_humanedit,
    "vitb32_2_humanedit": vitb32_2_humanedit,
}

# Ergebnisse speichern
results = {}
p_values = []

# Berechnung der T-Tests und p-Werte nur für "Validation"
for name, df in comparisons.items():
    results[name] = {}
    results[name]["Validation"] = {}
    for metric in vitb16_2_humanedit["Validation"].keys():
        t_stat, p_value = ttest_ind(vitb16_2_humanedit["Validation"][metric], df["Validation"][metric], equal_var=False)
        results[name]["Validation"][metric] = {"t-stat": t_stat, "p-value": p_value}
        p_values.append(p_value)

# Wende die Holm-Korrektur auf alle p-Werte an
rejected, pvals_corrected, _, _ = smm.multipletests(p_values, method='holm')

# Ausgabe der Ergebnisse mit korrigierten p-Werten
p_value_idx = 0  # Um den Index der p-Werte zu verfolgen

for name, categories in results.items():
    print(f"\n=== Vergleich: vitb16_2_humanedit vs {name} ===")
    for category, metrics in categories.items():
        if category == "Validation":  # Nur "Validation" ausgeben
            print(f"\n{category}:")
            for metric, values in metrics.items():
                # Holen des originalen und korrigierten p-Werts
                original_p_value = values['p-value']
                corrected_p_value = pvals_corrected[p_value_idx]
                
                # Ergebnisse ausgeben
                print(f"{metric}: t={values['t-stat']:.3f}, p={original_p_value:.5f}, p-corrected={corrected_p_value:.5f}")
                
                # Inkrementiere den Index für die p-Werte
                p_value_idx += 1



=== Vergleich: vitb16_2_humanedit vs vitb16_2_magicbrush ===

Validation:
loss: t=-627.805, p=0.00000, p-corrected=0.00000
mean_rank: t=-26.204, p=0.00000, p-corrected=0.00000
median_rank: t=-24.407, p=0.00000, p-corrected=0.00000
recall@1: t=6.790, p=0.00103, p-corrected=0.01445
recall@5: t=13.269, p=0.00001, p-corrected=0.00008
recall@10: t=17.594, p=0.00000, p-corrected=0.00000
recall@50: t=17.871, p=0.00000, p-corrected=0.00004

=== Vergleich: vitb16_2_humanedit vs vitb16_8_humanedit ===

Validation:
loss: t=6.025, p=0.00032, p-corrected=0.00484
mean_rank: t=2.461, p=0.03945, p-corrected=0.18860
median_rank: t=3.862, p=0.00483, p-corrected=0.05790
recall@1: t=-1.541, p=0.16208, p-corrected=0.18860
recall@5: t=-3.034, p=0.01632, p-corrected=0.13204
recall@10: t=-3.432, p=0.01320, p-corrected=0.13204
recall@50: t=-2.494, p=0.03772, p-corrected=0.18860

=== Vergleich: vitb16_2_humanedit vs vitb32_2_humanedit ===

Validation:
loss: t=-2.905, p=0.02060, p-corrected=0.13204
mean_rank: t

In [8]:
import json
import numpy as np
import matplotlib.pyplot as plt

json_files = ['fold1/history.json', 'fold2/history.json', 'fold3/history.json', 'fold4/history.json', 'fold5/history.json']
# List to store the validation data for plotting
validation_data = {'recall@1': [], 'recall@5': [], 'recall@10': [], 'recall@50': [],
                   'mean_rank': [], 'median_rank': []}

# Read each JSON file and collect the values for plotting
epochs = range(1, 12)  # Assuming there are 11 epochs in each dataset
for json_file in json_files:
    with open(json_file, 'r') as f:
        data = json.load(f)

        # Collect validation data
        validation_data['recall@1'].append(data['Validation']['recall@1'])
        validation_data['recall@5'].append(data['Validation']['recall@5'])
        validation_data['recall@10'].append(data['Validation']['recall@10'])
        validation_data['recall@50'].append(data['Validation']['recall@50'])
        validation_data['mean_rank'].append(data['Validation']['mean_rank'])
        validation_data['median_rank'].append(data['Validation']['median_rank'])

# Convert lists to numpy arrays for easier manipulation
validation_data = {key: np.array(value) for key, value in validation_data.items()}

# Plotting Recall Curves for Validation
fig, ax = plt.subplots(1, 2, figsize=(16, 6))  # 1 row, 2 columns

# Recall Curves
ax[0].plot(epochs, np.mean(validation_data['recall@1'], axis=0), label='Recall@1', color='green', marker='o')
ax[0].plot(epochs, np.mean(validation_data['recall@5'], axis=0), label='Recall@5', color='blue', marker='o')
ax[0].plot(epochs, np.mean(validation_data['recall@10'], axis=0), label='Recall@10', color='orange', marker='o')
ax[0].plot(epochs, np.mean(validation_data['recall@50'], axis=0), label='Recall@50', color='red', marker='o')
ax[0].set_title('Recall@k (Validation Split)', fontsize=18)
ax[0].set_xlabel('Epochs', fontsize=16)
ax[0].set_ylabel('Recall', fontsize=16)
ax[0].set_ylim(0, 100)  # Set y-axis limit for recall to range from 0 to 100
ax[0].tick_params(axis='both', labelsize=14)  # Increase tick font size
ax[0].legend(fontsize=15)

# Plotting Mean Rank and Median Rank Curves for Validation
ax[1].plot(epochs, np.mean(validation_data['mean_rank'], axis=0), label='Mean Rank', color='purple', marker='o')
ax[1].plot(epochs, np.mean(validation_data['median_rank'], axis=0), label='Median Rank', color='brown', marker='o')
ax[1].set_title('Mean Rank and Median Rank (Validation Split)', fontsize=18)
ax[1].set_xlabel('Epochs', fontsize=16)
ax[1].set_ylabel('Rank', fontsize=16)
ax[1].set_ylim(0, max(np.mean(validation_data['mean_rank'], axis=0)) + 10)  # Ensure y-axis starts from 0 for rank
ax[1].tick_params(axis='both', labelsize=14)  # Increase tick font size
ax[1].legend(fontsize=15)

plt.tight_layout()
plt.savefig("recall_rank_curves.png", bbox_inches='tight')
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'fold1/history.json'

In [5]:
print(1)

1
